In [1]:
# !sudo apt-get install python3.7
# !sudo apt-get update -y
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
# !sudo update-alternatives --config python3

In [1]:
!pip install numpy
!pip install pyopenms
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import re

EXPERIMENT_NAME = "N4-1A"
# reading file
file1 = open(f'./{EXPERIMENT_NAME}.txt', 'r')
oligo_list = [line for line in file1.readlines() if re.search("^[a-zA-Z][1-9]", line)]
import csv

header = ['where\tcode\tseq\tDMT\tcomment\tlength\t']
data = [row.strip() for row in oligo_list]


with open('./oligos.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    writer = csv.writer(f, quotechar='\"', delimiter='\n', quoting=csv.QUOTE_NONE, escapechar='\\')
    # write the data
    writer.writerow(data)

In [3]:
import pandas as pd

df = pd.read_csv('./oligos.csv', delimiter='\t')
# drop NaN seq
df = df[df['seq'].notna()]
oligo_list = df['seq'].to_list()

In [4]:
from Bio.Seq import Seq
from Bio.SeqUtils import molecular_weight

# Bio.SeqUtils.molecular_weight(seq, seq_type='DNA', double_stranded=False, circular=False, monoisotopic=False)

from pyopenms import *

# # reading file
# file1 = open('./OVFF317_oligos_seq.txt', 'r')
# oligo_list = file1.readlines()

mass_list=[]
# https://biopython.org/docs/1.75/api/Bio.SeqUtils.html
for oligo_seq in oligo_list: 
  try:
    my_dna = Seq(oligo_seq)
  except:
    print(oligo_seq)
  try:
    print(f"{oligo_seq.strip()}\t" + "%0.2f" % (molecular_weight(my_dna)-79.97))
    mass_list+=["%0.2f" % (molecular_weight(my_dna)-79.97)]
  except ValueError:
    print(f"{oligo_seq.strip()}\t" + "'N' is not a valid unambiguous letter for DNA")
    mass_list+=["NaN"]

ATC TGT ATC TTA AAG ACT ATC ATC AAG ACT CAC ATC AAG ACT ATC ATC AAG TCA TAC	17390.25
TAT ATC TTG ATG ATT ACC TAA AGA GGA GAT GAT TGA TGT ATG ACT TGA TGA TAG TCT TGA TG	19269.41
TCC TCT TTA GGT AAT CAT CAA GAT ATA CGG TAA GAC TAT CAT CAA GAC TAA CAT CAA GTC AAG ACC	20240.07
GCT TAT TGA TTG GCT TAT GGT CTA TCC TTT AGA TAG GTC TTG ACT TGA TGT TAG TCT	17572.26
GAC CAT AAG CCA ATC AAT AAG CCC ACA CTA TAA GGA GGA GAC AAA CAG AGA	15738.21
GAC TAT CTC TTA TAG TTC TCC TCT TAT AGT CTC TGT TTG TCT CCT CCT T	14812.50
ATA AGA GGA GAA CTA TAA GAG ATA GTC ACT AAC AAA CAA TTA TAA TAA GAA TAA T	17064.11
AAG ATA GTC TTT AAG TCT ATG ATT GTA ATA GTA TTA TTC TTA TTA TAA TTG TTT GTT AGT	18531.96
ATT ACA ATC ATA GAC TTA AAG ACT ATC TTA AAG ATT ATC TCA AAA TAC CTA TTG ACT AT	18060.71
CCG TCT CGA TGG AAT GAA TAA TAC ATA GAC CAG ACA CCA TAG TCA ATA GGT ATT TTG AGA TAA TCT	20351.13
TTA TTC ATT CCA TCG AGA CGG GAC ATA GAG ACC TTA CCC GAA AGG AAG T	15097.74
TGC TAC TTA ACG TTA TCA GAC TTA GTG ACT TCC TTT CGG GTA A

In [5]:
df['monoisotopic_mass']=mass_list

In [6]:
export_dataframe=df[['where', 'code', 'length', 'seq', 'monoisotopic_mass']]

In [7]:
export_dataframe.to_csv(f'./{EXPERIMENT_NAME}_calculated.csv', sep = '\t', index=False)

In [8]:
export_dataframe


,where,code,length,seq,monoisotopic_mass
0,A1,VibriophageN4_F1-1,#= 57,ATC TGT ATC TTA AAG ACT ATC ATC AAG ACT CAC AT...,17390.25
1,B1,VibriophageN4_F1-2,#= 62,TAT ATC TTG ATG ATT ACC TAA AGA GGA GAT GAT TG...,19269.41
2,C1,VibriophageN4_F1-3,#= 66,TCC TCT TTA GGT AAT CAT CAA GAT ATA CGG TAA GA...,20240.07
3,D1,VibriophageN4_F1-4,#= 57,GCT TAT TGA TTG GCT TAT GGT CTA TCC TTT AGA TA...,17572.26
4,E1,VibriophageN4_F1-5,#= 51,GAC CAT AAG CCA ATC AAT AAG CCC ACA CTA TAA GG...,15738.21
...,...,...,...,...,...
379,L24,VibriophageN4_F9-2,#= 54,TTT GAT GTA CTG TTC AAA ATC CCA AGG GCG ATA GG...,16545.64
380,M24,VibriophageN4_F9-3,#= 42,GGG ATT TTG AAC AGT ACA TCA AAG ACC TTG AAG CT...,13001.39
381,N24,VibriophageN4_F9-4,#= 44,TGG AAC ACA ATC ACA CCA CCC TCG GCT GCA CAA GC...,13399.64
382,O24,VibriophageN4_F9-5,#= 54,GTG GTG TGA TTG TGT TCC ATA ACG GAA TCA AGT AT...,16714.75


In [9]:
molecular_weight(Seq("TG"))

651.4144

In [10]:
print([ord(char) - 96 for char in input('Write Text: ').lower()])

Write Text: 
[]


In [11]:
seq = "CCCGGGGTACCACGGAGAGATGGTGGAAATCATCTTTCTCGTGGGCATCCTTGATGGCCACCTCGTCGGAAGTGCCCACGAGGATGACAGCAATGCCAATGCTGGGGGGGCTCTTCTGAGAACGAGCTCTGCTGCCTGACACGGCCAGGACGGCCAACACCAACCAGAACTTGGGAGAACAGCACTCCGCTCTGGGCTTCATCTTCAACTCGTCGACTCCCTGCAAACACAAAGAAAGAGCATGTTAAAATAGGATCTACATCACGTAACCTGTCTTAGAAGAGGCTAGATACTGCAATTCAAGGACCTTATCTCCTTTCATTGAGCACCAAACCCAACTCCATCTACCAGCCTACTCTCTTATCTCTGGTATT"


In [12]:
seq.upper()

'CCCGGGGTACCACGGAGAGATGGTGGAAATCATCTTTCTCGTGGGCATCCTTGATGGCCACCTCGTCGGAAGTGCCCACGAGGATGACAGCAATGCCAATGCTGGGGGGGCTCTTCTGAGAACGAGCTCTGCTGCCTGACACGGCCAGGACGGCCAACACCAACCAGAACTTGGGAGAACAGCACTCCGCTCTGGGCTTCATCTTCAACTCGTCGACTCCCTGCAAACACAAAGAAAGAGCATGTTAAAATAGGATCTACATCACGTAACCTGTCTTAGAAGAGGCTAGATACTGCAATTCAAGGACCTTATCTCCTTTCATTGAGCACCAAACCCAACTCCATCTACCAGCCTACTCTCTTATCTCTGGTATT'

In [13]:
len(seq)
my_dna = Seq(seq)
molecular_weight(my_dna)

115174.42379999981